# GridClim analysis

In [ ]:
# Small helper lib.
import attribution.funcs
import attribution.bootstrap

# Others.
import iris
import iris.coord_categorisation
import iris.plot as iplt
import iris_utils
from functools import partial
from matplotlib import pyplot as plt
import numpy as np
import scipy.stats as scstats
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from climix.metadata import load_metadata
import dask
from dask.distributed import Client
import os
import glob
import scipy
import pandas as pd
import geopandas as gpd

In [ ]:
data_path = "/nobackup/rossby26/users/sm_erhol/extremeEventAttribution/"

In [ ]:
client = Client(processes=True, threads_per_worker=1)
# client

In [ ]:
client.scheduler_info

In [ ]:
# Get the sweref projection.
sweref = ccrs.epsg(3006)

In [ ]:
# This file contains shapes of most countries in the world.
# https://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-admin-0-boundary-lines/
fname = "/home/sm_erhol/data/ne_10_admin_0_countries/ne_10m_admin_0_countries.shp"

In [ ]:
gdf = gpd.read_file(fname)

In [ ]:
# Select Sweden.
swe_shapes = gdf[gdf.SOVEREIGNT == "Sweden"].geometry
swe_mainland = swe_shapes.iloc[0].geoms[0]

## Load the data
Let's load the SweGridClim data.

In [ ]:
base_path = "/nobackup/smhid17/proj/sik/SMHIGridClim_NORDIC-11/v0.9/netcdf/day/pr/"

In [ ]:
# This gives a list of files in the base path matchig the wildcard.
files = glob.glob(base_path + "*.nc")

In [ ]:
cube = iris.load(files)

We want to concatenate these cubes to one.
But have to remove some attributes first.

In [ ]:
removed = iris.util.equalise_attributes(cube)

Now we should hopefully be able to concatenate.

In [ ]:
# We concat on time.
cube = cube.concatenate_cube()
time_constraint = iris.Constraint(time=lambda cell: 1971 <= cell.point.year <= 2018)
# Extract the relevant time period.
cube = cube.extract(time_constraint)

In [ ]:
cube

Extract data for Sweden

## Region selection
We probably don't want to look over all of Sweden.
Which region should we select the data over?
Some box around Gävle, where data should be homogeneous.

Could make an average map and use this to select an area around POI?

In [ ]:
clim_cube = cube.collapsed("time", iris.analysis.MEAN)

In [ ]:
mask_points = [[14.5, 14.5, 19.5, 19.5], [57.7, 61.2, 57.7, 61.2]]

In [ ]:
# Gävle point
lat = 60.73284099330242
lon = 17.09885344649177
fig, ax = plt.subplots(figsize=(7, 9), subplot_kw={"projection": sweref})
iplt.contourf(clim_cube, 30, axes=ax)
ax.scatter([lon], [lat], s=50, transform=ccrs.PlateCarree(), label="Gävle")
ax.scatter(
    mask_points[0],
    mask_points[1],
    s=50,
    c="k",
    transform=ccrs.PlateCarree(),
    label="Box corners",
)

ax.coastlines()
ax.legend()
ax.set_title("Average precipitation flux");

We then have to convert the coordinates to the CoordSystem of our cube.

In [ ]:
# Get the coord system of the cube. Convert it to cartopy.
target_projection = cube.coord_system().as_cartopy_projection()

In [ ]:
# Convert mask points to ndarray
points = np.asarray(mask_points)
# Transform them to the cube projection.
transformed_points = target_projection.transform_points(
    ccrs.PlateCarree(), points[0, :], points[1, :]
)

In [ ]:
# Save the transformed coordinates of the bounding box.
np.save(
    os.path.join(data_path, "etc/region_points_transformed.npy"),
    transformed_points,
)

Create a constraint from the converted corner coordinates.

In [ ]:
# Create the constraint.
region_constraint = iris.Constraint(
    grid_latitude=lambda v: transformed_points[:, 1].min()
    < v
    < transformed_points[:, 1].max(),
    grid_longitude=lambda v: transformed_points[:, 0].min()
    < v
    < transformed_points[:, 0].max(),
)

In [ ]:
# And extract the region.
reg_cube = cube.extract(region_constraint)

In [ ]:
# reg_cube

Look at the selected data.

In [ ]:
# Gävle point
lat = 60.73284099330242
lon = 17.09885344649177
fig, ax = plt.subplots(figsize=(7, 9), subplot_kw={"projection": sweref})
iplt.contourf(reg_cube[0, :, :], 30, axes=ax)
ax.scatter([lon], [lat], s=50, transform=ccrs.PlateCarree(), label="Gävle")
ax.scatter(
    mask_points[0],
    mask_points[1],
    s=20,
    c="k",
    transform=ccrs.PlateCarree(),
    label="Box corners",
)

ax.coastlines()
ax.legend()
# Set the extent to put the data into context.
ax.set_extent([10, 20, 50, 75], crs=ccrs.PlateCarree())

## Rx1 annual
### Event definition
It rained 161 mm in 24 hours in Gävle during the event.
This gives us a threshold for the event

In [ ]:
# [mm s or kg/m2/s]
threshold = 161

This however raises the question, is it a fair comparison to take the daily intensity of the gridded product and compare it to station data like this?

We will use Climix to calculate the index we are interested in.

In [ ]:
# index catalog
catalog = load_metadata()

In [ ]:
rx1_ann_index = catalog.prepare_indices(["rx1day"])[0]

In [ ]:
# Can't have a year coordiante when passing to climix.
try:
    reg_cube.remove_coord("year")
except iris.exceptions.CoordinateNotFoundError:
    pass
rx1_ann = rx1_ann_index([reg_cube], client)

In [ ]:
# We mask the data with sweden.
mask = iris_utils.mask_from_shape(
    rx1_ann, swe_mainland, coord_names=["grid_latitude", "grid_longitude"]
)
iris_utils.mask_cube(rx1_ann, mask)

### Fitting an extreme value distribution to Rx1
Now we can start looking at the extremes, e.g. annual Rx1.
In this case Rx1 should simply be the annual max?
Since we already have daily values.

In [ ]:
rx1_ann.data.compressed().shape

In [ ]:
# Note, density is way above one since the bin values are so small.
# e.g. the widht of each bin is ~0.0001, hence integrating to 1
plt.hist(rx1_ann.data.compressed());

We try and fit a number of extreme value distributions to the data.

In [ ]:
# Some distributions describing extremes.
dists = {
    "genextreme": scstats.genextreme,
    "genpareto": scstats.genpareto,
    "gamma": scstats.gamma,
    "gengamma": scstats.gengamma,
    "gumbel_l": scstats.gumbel_l,
    "gumbel_r": scstats.gumbel_r,
}
# data
data = rx1_ann.data.compressed()

Before we do the bootstrap, we want to check the goodness of fit for the distribution and the data.
For this we use a Kolmogorov-Smirnof test (KS-test).
For a goodness of fit this is a bit unintuitive.
The 0-hypothesis is that the distributions are the same, hence we are looking for a high p-value here. e.g. that we can't say that the dists are different.

In [ ]:
# Fit each distribution and evaluate KS test.
for key, dist in dists.items():
    fit = dist.fit(data)
    print(f"{key}:", scstats.ks_1samp(data, dist.cdf, args=fit))

In [ ]:
# Note, density is way above one since the bin values are so small.
x = np.linspace(0, 120, 200)
# e.g. the widht of each bin is ~0.0001, hence integrating to 1
plt.hist(rx1_ann.data.compressed(), bins=20, density=True)
for key, dist in dists.items():
    fit = dists[key].fit(data)
    plt.plot(x, dists[key].pdf(x, *fit), label=key)
plt.legend()

For a KS-test high p-value = we can't reject the null hypothesis that they are from the same distributions.

$\rightarrow$ the GEV distribution has the better fit.

### Regression to GMST
To scale the above distribution with the use of GMST we first need to fit a regression between the Rx1 and GMST.
The slope of the regression can then be used for the scaling.

But first we load the GISTEMP data from NASA.

In [ ]:
gmst_path = os.path.join(data_path, "etc/gistemp.txt")

In [ ]:
# This gives us the smoothed gmst data  for the timespan
# covered by the cube.
gmst_data = attribution.funcs.get_gmst(rx1_ann, path=gmst_path)

In [ ]:
# Lets get the data of the rx1 cube.
rx1_ann_data = np.zeros((rx1_ann.shape[0], rx1_ann.data[0, :, :].compressed().shape[0]))
# We need to compress the data for each year. This has to be done
# in a loop I think.
for i, year in enumerate(rx1_ann.data):
    rx1_ann_data[i] = year.compressed()

In [ ]:
# Check that first dimensions match.
assert rx1_ann_data.shape[0] == gmst_data.shape[0]

In [ ]:
# Uncomment to look at the data.
# fig, ax = plt.subplots(figsize=(7, 7))
# ax.scatter(np.broadcast_to(gmst_data, rx1_ann_data.shape).flatten(),
#                             rx1_ann_data.flatten(), s=5);
# ax.set_xlabel("GMST")
# ax.set_ylabel("Precipitation intensity");
# ax.set_title("Pooled region scatter");

In [ ]:
# For the linear regression we use Sklearn.
from sklearn.linear_model import LinearRegression

In [ ]:
# This can make clever use of the multiregression feature, we want
# know the regression for each point.
reg = LinearRegression().fit(gmst_data, rx1_ann_data)

In [ ]:
reg.coef_.shape

In [ ]:
rx1_ann_data.shape

In [ ]:
# We broadcast the slopes to have a slope for each entry in the pooled data.
slopes_broad = np.broadcast_to(reg.coef_.reshape(1, -1), rx1_ann_data.shape)

In [ ]:
slopes_broad = slopes_broad.flatten()

In [ ]:
# These should now have the same shape.
assert slopes_broad.shape == data.shape

We scale the distribution by making the location and scale a function of the temperature anomaly, using the slope of the regression.

$\mu = \mu_0 \mathrm{exp}(\alpha T' / \mu_0),\, \sigma = \sigma_0\mathrm{exp}(\alpha T'/ \mu_0)$

This is implemented in the `attribution.scale_dist_params` which is used by `calc_prob_ratio` available in `attribution.funcs`.

### Probabilities
The bootstrap takes the variation in regression slope into account.
For the resampling we are randomly selecting the regression coefficient for each resample.
The jackknife works by leaving out the slope corresponding to the sample (day) that is left out.

Randomly picking slopes should result in a good representation since the regression coefficients are normally distributed.

In [ ]:
plt.hist(reg.coef_);

In [ ]:
reload(attribution.funcs)

In [ ]:
# We need a random number generator.
rng = np.random.default_rng()

Create some test data.
Only used for testing.

In [ ]:
test_indices = rng.integers(0, data.shape[0], 7000)

In [ ]:
test_data = data[..., test_indices]
test_slopes = slopes_broad[..., test_indices]

Create a partial function which we can pass to the bootstrap.

In [ ]:
# Create a partial function of calc_prob_ratio which can be passed
# to the bootstrap.
# temperature indicates to which temperature we scale the counterfactua
# climate. In this case we want a climate that is 1.2 degrees colder.
calc_prob_ratio_p = partial(
    attribution.funcs.calc_prob_ratio,
    threshold=threshold,
    temperature=-1.2,
    dist=dists["genextreme"],
)

First we calculate the probability ratio of the sample

In [ ]:
# The probability ratio of the sample.
rx1_ann_pbr = calc_prob_ratio_p(test_data, test_slopes)

rx1_ann_pbr

In [ ]:
reload(attribution.bootstrap)

In [ ]:
# Compute the bootstrapped CI of the probability ratio
# Note that this is much faster on an mp pool. 
rx1_ann_pbr_ci, rx1_ann_pbr_med, theta_hat_b = attribution.bootstrap.bootstrap_mp(
    (data, slopes_broad), calc_prob_ratio_p, n_resamples=9999, batch=1, client=client
)

In [ ]:
plt.hist(theta_hat_b);

The probability ratio(s) (PR) for an event the magnitude of the Gävle

In [ ]:
rx1_ann_pbr_ci

In [ ]:
prob_ratios_ci = np.asarray(
    [
        rx1_ann_pbr_ci.confidence_interval.low,
        rx1_ann_pbr_med,
        rx1_ann_pbr_ci.confidence_interval.high,
    ]
)

In [ ]:
prob_ratios_ci

In [ ]:
np.save(os.path.join(data_path, "etc/rx1-ann_pbr_gridclim"), prob_ratios_ci)

In [ ]:
# np.load(os.path.join(data_path, "etc/rx1-ann_pbr_gridclim.npy"))

Since the PR CI include 1 we cannot make a attribution statement for this event.

## Rx1 seasonal
### Event definition
- It rained 161 mm in 24 hours in Gävle during the event.

In [ ]:
# [mm s or kg/m2/s]
threshold = 161

which can define as the event to look for.

We can do this quickly in the whole of GridClim

This however raises the question, is it a fair comparison to take the daily intensity of the gridded product and compare it to station data like this?

We will use Climix to calculate the index we are interested in.

In [ ]:
# index catalog
# Make sure that default is changed in the yml file.
catalog = load_metadata()

In [ ]:
rx1_seasonal_index = catalog.prepare_indices(["rx1day"])[0]

In [ ]:
# Can't have a year coordiante when passing to climix.
try:
    reg_cube.remove_coord("year")
except iris.exceptions.CoordinateNotFoundError:
    pass
try:
    reg_cube.remove_coord("season_year")
except iris.exceptions.CoordinateNotFoundError:
    pass
try:
    reg_cube.remove_coord("season")
except iris.exceptions.CoordinateNotFoundError:
    pass
rx1_seasonal = rx1_seasonal_index([reg_cube], client)

In [ ]:
# We mask the data with sweden.
mask = iris_utils.mask_from_shape(
    rx1_seasonal, swe_mainland, coord_names=["grid_latitude", "grid_longitude"]
)
iris_utils.mask_cube(rx1_seasonal, mask)

We select only JJA for now.

In [ ]:
constraint = iris.Constraint(season="jja")
rx1_jja = rx1_seasonal.extract(constraint)

### Fitting an extreme value distribution to Rx1
Now we can start looking at the extremes, e.g. annual Rx1.
In this case Rx1 should simply be the annual max?
Since we already have daily values.

In [ ]:
# Note, density is way above one since the bin values are so small.
# e.g. the widht of each bin is ~0.0001, hence integrating to 1
plt.hist(rx1_jja.data.compressed());
plt.hist(rx1_ann.data.compressed(), alpha=0.5);

We try and fit a number of extreme value distributions to the data.

In [ ]:
# Get the GEV dist object
dists = {
    "genextreme": scstats.genextreme,
    "genpareto": scstats.genpareto,
    "gamma": scstats.gamma,
    "gengamma": scstats.gengamma,
    "gumbel_l": scstats.gumbel_l,
    "gumbel_r": scstats.gumbel_r,
}
# data
data = rx1_jja.data.compressed()

Before we do the bootstrap, we want to check the goodness of fit for the distribution and the data.
For this we use a Kolmogorov-Smirnof test (KS-test).

In [ ]:
for key, dist in dists.items():
    fit = dist.fit(data)
    print(f"{key}:", scstats.ks_1samp(data, lambda x: dist.cdf(x, *fit)))

In [ ]:
# Note, density is way above one since the bin values are so small.
x = np.linspace(0, 120, 200)
# e.g. the widht of each bin is ~0.0001, hence integrating to 1
plt.hist(data, density=True)
for key, dist in dists.items():
    fit = dist.fit(data)
    plt.plot(x, dist.pdf(x, *fit), label=key)
plt.legend()

For a KS-test high p-value = we can't reject the null hypothesis that they are from different distributions.
Thus, the GEV has the best performance, even if it is not great.

### Regression to GMST
To scale the above distribution with the use of GMST we first need to fit a regression between the Rx1 and GMST.
The slope of the regression can then be used for the scaling.

But first we load the GISTEMP data from NASA.

In [ ]:
gmst_path = os.path.join(data_path, "etc/gistemp.txt")

In [ ]:
reload(attribution.funcs)

In [ ]:
# This gives us the smoothed gmst data  for the timespan
# covered by the cube.
gmst_data = attribution.funcs.get_gmst(rx1_jja, path=gmst_path)

In [ ]:
# Lets get the data of the rx1 cube.
rx1_jja_data = np.zeros((rx1_jja.shape[0], rx1_jja.data[0, :, :].compressed().shape[0]))
# We need to compress the data for each year. This has to be done
# in a loop I think.
for i, year in enumerate(rx1_jja.data):
    rx1_jja_data[i] = year.compressed()

In [ ]:
# Check that first dimensions match.
assert rx1_jja_data.shape[0] == gmst_data.shape[0]

In [ ]:
# For the linear regression we use Sklearn.
from sklearn.linear_model import LinearRegression

In [ ]:
# This can make clever use of the multiregression feature, we want
# know the regression for each point.
reg = LinearRegression().fit(gmst_data, rx1_jja_data)

In [ ]:
# We broadcast the slopes to have a slope for each entry in the pooled data.
slopes_broad = np.broadcast_to(reg.coef_.reshape(1, -1), rx1_jja_data.shape)

In [ ]:
slopes_broad = slopes_broad.flatten()

In [ ]:
# These should now have the same shape.
assert slopes_broad.shape == data.shape

### Probabilities

In [ ]:
test_indices = rng.integers(0, data.shape[0], 4000)

In [ ]:
test_data = data[..., test_indices]
test_slopes = slopes_broad[..., test_indices]

In [ ]:
fit = dists["genextreme"].fit(rx1_jja.data.compressed())

In [ ]:
1 - dists["genextreme"].cdf(161, *fit)

In [ ]:
reload(attribution.funcs)

In [ ]:
# Create a partial function of calc_prob_ratio which can be passed
# to the bootstrap.
# temperature indicates to which temperature we scale the counterfactua
# climate. In this case we want a climate that is 1.2 degrees colder.
calc_prob_ratio_p = partial(
    attribution.funcs.calc_prob_ratio,
    threshold=50,
    temperature=-1.2,
    dist=dists["genextreme"],
)

Calculate the probability ratio CI, median and theta_hat_b

In [ ]:
reload(attribution.bootstrap)

In [ ]:
# Compute the bootstrapped CI of the probability ratio
rx1_jja_pbr_ci, rx1_jja_pbr_med, theta_hat_b = attribution.bootstrap.bootstrap_mp(
    (test_data, test_slopes), calc_prob_ratio_p, n_resamples=1000, batch=1, client=client
)

In [ ]:
plt.hist(theta_hat_b);

In [ ]:
rx1_jja_pbr_ci

The probability ratio(s) (PR) for an event the magnitude of the Gävle

In [ ]:
prob_ratios_ci

The probability ratio(s) (PR) for an event the magnitude of the Gävle

In [ ]:
np.save(os.path.join(data_path, "etc/rx1-jja_prb_gridclim"), prob_ratios)

Since the PR CI include 1 we cannot make a attribution statement for this event.

## Rx2
First we have to calculate the Rx2 index for the GridClim data.
This can be done using the Climix package.

In [ ]:
catalog = load_metadata()

In [ ]:
rx2_ann_index = catalog.prepare_indices(["rx2day"])[0]

In [ ]:
# The cube can't have a year coordinate.
try:
    reg_cube.remove_coord("year")
except iris.exceptions.CoordinateNotFoundError:
    pass
# Calculate the index.
rx2_ann = rx2_ann_index([reg_cube], client)

In [ ]:
# We mask the data with sweden.
mask = iris_utils.mask_from_shape(
    rx2_ann, swe_mainland, coord_names=["grid_latitude", "grid_longitude"]
)
iris_utils.mask_cube(rx2_ann, mask)

In [ ]:
iplt.contourf(rx2_ann[0, :, :, :])

### Event definition
The Gävle event was a single day event, thus a clear definition is not available.
So what should we pick here? 

- Based on percentiles? 

### Fitting an extreme event distribution

In [ ]:
rx2_ann_data = rx2_ann.data.compressed()

In [ ]:
rx2_ann_data.max()

In [ ]:
plt.hist(rx2_ann_data);

We try and fit a number of extreme value distributions to the data.

In [ ]:
# Get the GEV dist object
dists = {
    "genextreme": scstats.genextreme,
    "genpareto": scstats.genpareto,
    "gamma": scstats.gamma,
    "gengamma": scstats.gengamma,
    "gumbel_l": scstats.gumbel_l,
    "gumbel_r": scstats.gumbel_r,
}

Before we do the bootstrap, we want to check the goodness of fit for the distribution and the data.
For this we use a Kolmogorov-Smirnof test (KS-test).

In [ ]:
for key, dist in dists.items():
    fit = dist.fit(rx2_ann_data)
    print(f"{key}:", scstats.ks_1samp(rx2_ann_data, lambda x: dist.cdf(x, *fit)))

In [ ]:
# Note, density is way above one since the bin values are so small.
x = np.linspace(0, 120, 200)
# e.g. the widht of each bin is ~0.0001, hence integrating to 1
plt.hist(rx2_ann_data, bins=20, density=True)
for key, dist in dists.items():
    fit = dists[key].fit(rx2_ann_data)
    plt.plot(x, dists[key].pdf(x, *fit), label=key)
plt.legend()

For a KS-test high p-value = we can't reject the null hypothesis that they are from different distributions.
Thus, the GEV has the best performance, even if it is not great.

### Regression to GMST
To scale the above distribution with the use of GMST we first need to fit a regression between the Rx1 and GMST.
The slope of the regression can then be used for the scaling.

But first we load the GISTEMP data from NASA.

In [ ]:
gmst_path = os.path.join(data_path, "etc/gistemp.txt")

In [ ]:
# This gives us the smoothed gmst data  for the timespan
# covered by the cube.
gmst_data = attribution.funcs.get_gmst(rx2_ann, path=gmst_path)

In [ ]:
# Lets get the data of the rx1 cube.
# Reshape to flatten the spatial dimensions.
rx2_ann_data_reg = rx2_ann.data.reshape(rx2_ann.shape[0], -1)

In [ ]:
# Check that first dimensions match.
assert rx2_ann_data_reg.shape[0] == gmst_data.shape[0]

In [ ]:
# Uncomment to look at the data.
# fig, ax = plt.subplots(figsize=(7, 7))
# ax.scatter(np.broadcast_to(gmst_data, rx1_ann_data.shape).flatten(),
#                             rx1_ann_data.flatten(), s=5);
# ax.set_xlabel("GMST")
# ax.set_ylabel("Precipitation intensity");
# ax.set_title("Pooled region scatter");

In [ ]:
# For the linear regression we use Sklearn.
from sklearn.linear_model import LinearRegression

In [ ]:
# This can make clever use of the multiregression feature, we want
# know the regression for each point.
reg = LinearRegression().fit(gmst_data, rx2_ann_data_reg)

We scale the distribution by making the location and scale a function of the temperature anomaly, using the slope of the regression.

$\mu = \mu_0 \mathrm{exp}(\alpha T' / \mu_0),\, \sigma = \sigma_0\mathrm{exp}(\alpha T'/ \mu_0)$

This is implemented in the `attribution.scale_dist_params`

In [ ]:
# Create current climate dists with CI
dists_ci = [dists["genextreme"](*fit) for fit in fits_ci]

In [ ]:
all_scaled_dists = attribution.funcs.scale_distributions(
    fits_ci, reg, dists["genextreme"]
)

In [ ]:
attribution.plotting.plot_distribution(
    data, dists_ci, all_scaled_dists, title="Rx2 GridClim"
)

### Probabilities

The probability ratio(s) (PR) for an event the magnitude of the Gävle

In [ ]:
prob_ratios = attribution.funcs.get_probability_ratios(
    dists_ci, all_scaled_dists, threshold
)

In [ ]:
np.save(os.path.join(data_path, "etc/rx2-ann_prb_gridclim"), prob_ratios)

In [ ]:
prob_ratios

Since the PR CI include 1 we cannot make a attribution statement for this event.

## Next step

[Analysing EOBS](./eobs.ipynb)